In [4]:
# prepare data for training 
# data >> numpy array >> torch.*Tensor
# for images: e.g. Pillow, OpenCV
# for audio: scipy,librosa
# for text: raw Python, Cython,NLTK and SpaCy 
# package torchvision: data loaders for common dataset 

# load and normalizing CIFAR10 with torchvision
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
# the output of torchvision dataset is PILImage images of range [0,1]
# we transform them to tensors of normalized size [-1,1]

# compose two transformation as one 
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)
# tuble 
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [ ]:
# show some images
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
# define a Convolutional Neural Network 
# copy Neural Network defined before and modify to 3 channel 
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
# for using GPU, do net.cuda()

In [ ]:
# use cross-entropy loss and SGD with momentum 
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.001,momentum=0.9)

In [ ]:
# train the network
# loop over data iterator and feed the inputs to the network
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)
        # for using GPU
        # do inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # i = 1999
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


In [ ]:
# test the network on the test data 
# display an image 
dataiter = iter(testloader)
images,labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ',' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
# the ouputs are energies for 10 classes
outputs = net(Variable(images))

_,predicted = torch.max(outputs.data,1)
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

In [ ]:
# look the performance of NN on whole dataset
correct = 0 # correctly classified num
total = 0
for data in testloader:
    images,labels = data
    outputs = net(Variable(images))
    _,predicted = torch.max(outputs.data,1) # predicted class 
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('Accuracy of the network on the 10000 test images: %d %%' %
     (100*correct/total))

In [ ]:
# check well classified classes and badly classified classes
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images,labels = data
    outputs = net(Variable(images))
    _,predicted = torch.max(outputs.data,1)
    c = (predicted == labels).squeeze() # correct classified
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] +=1
        
for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
    classes[i],100*class_correct[i]/class_total[i]))